In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *
#------ Path towards the required folders
expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/' # a possible directory for exp data
g0simdir   = home_dir+'/g0_sim/'        # This is the main directory of g0 simulations
simname    = '48x32x16x16x6_nunifvg'    # This is the name of the simulation
simdir     = 'gk_tcv_posD_iwl_3x2v_AH/' # This is the simulation directory relative to the g0sim
fileprefix = 'gk_tcv_posD_iwl_3x2v_D02' # This is the prefix before the '-*.gkyl'
wkdir      = 'wk/'                      # This is where the .gkyl files sit relatively to the sim dir
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                 # Elongation factor
    delta       = 0.3,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)

# Setup the data parameters, receipe and normalization of the fields
simulation.set_data_param(
    expdatadir = expdatadir, g0simdir = g0simdir, simname = simname, 
    simdir = simdir, fileprefix = fileprefix, wkdir = wkdir, 
    species = simulation.species)

# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

#----- set up the sources
simulation.set_OMPsources(
    n_srcOMP=2.4e23,
    x_srcOMP=0.0,
    Te_srcOMP=2 * simulation.species['elc'].T0,
    Ti_srcOMP=2 * simulation.species['ion'].T0,
    sigma_srcOMP=0.03 * simulation.geom_param.Lx,
    floor_src=1e-2,
)

# Check available frames of the ES potential
tfs = find_available_frames(simulation,'field')
print(tfs)

# ---- Set up normalization and units
simulation.normalize('t','mus')
# simulation.normalize('x','minor radius')
# simulation.normalize('y','Larmor radius')
simulation.normalize('z','pi')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','beta')
simulation.normalize('energies','MJ')

In [ ]:
from classes import Frame
from tools import math_tools
from scipy.interpolate import interp1d
f_ = Frame(simulation,'ne',tf=10)
f_.load()

#. Give human readable symboles for useful variables
Fxyz = f_.values

#. Load the new_grids (1 element less than nodes)
x    = f_.new_grids[0]; Nx = len(x)
y    = f_.new_grids[1]; Ny = len(y)
z    = f_.new_grids[2]; Nz = len(z)

# Densification factor and new number of points in z
Nfacz  = 1
Nzdens = Nfacz * Nz
# Create denser z grid
zdens = np.linspace(z[0], z[-1], Nzdens)
# Create an empty array for the interpolated data
Fxyz_dens = np.zeros((Nx, Ny, Nzdens))
# Interpolate over the z-axis for each (x, y) pair

for i in range(Nx):
    for j in range(Ny):
        # Define the 1D interpolation along the z-axis
        interp_func = interp1d(z, Fxyz[i, j, :], kind='linear', axis=-1, fill_value="extrapolate")
        # Interpolate to the new denser z grid
        Fxyz_dens[i, j, :] = interp_func(zdens)

[Xxyz, Yxyz, Zxyz] = math_tools.custom_meshgrid(x,y,zdens)

Ny   = len(y)
# y-y0 array (named u for shorter symbols)
u    = y - y[0]
Ly   = u[-1]
dy   = y[1] - y[0]

[Xxuz, Uxuz, Zxuz] = math_tools.custom_meshgrid(x,u,zdens)

#.Magnetic safety factor in the middle of the simulation box
qprofile = simulation.geom_param.qprofile
r_x      = simulation.geom_param.r_x
r0 = simulation.geom_param.r0
q0 = qprofile(r0)

xLCFS = simulation.geom_param.x_LCFS
ixLCFS = math_tools.closest_index(x,xLCFS)

#.Composed geometry factors
n0       = 2.*np.pi*r0/q0/Ly # toroidal mode number
Cy       = n0*Ly/(2*np.pi)
Phi_xuz  = qprofile(Xxuz)*Zxuz - 2*np.pi/n0 *  Uxuz/Ly
qchimphi = u/Cy
#.Fourier transform the field we want to treat along y, 
# we do full FFT and not real to have same dimensions
Fxkyz   = np.fft.fft(Fxyz_dens, axis=1, norm="forward")
ky      = np.fft.fftfreq(Ny, d=dy)  # Fourier frequencies for the y-dimension (ky)
Nky     = len(ky)
[Xxkyz,KYxkyz,Zxkyz] = math_tools.custom_meshgrid(x,ky,zdens)

#.Check the TS BC from expression 3.29 in Goerler thesis
# F(x,ky,z+Lz) = F(x,ky,z)*exp(-2*pi*n0*q(x)*j)
# where we replaced j as Cy*ky/n0
Fxkyz_TS = Fxkyz * np.exp(-2*np.pi*qprofile(Xxkyz)*Cy*KYxkyz)
# Now Fxkyz_TS@z=0 must be equal Fxkyz@z=Lz
print('Fxkyz(z=Lz) = %f, Fxkyz_TS(z=0) = %f'%(np.real(Fxkyz_TS[1,0,-1]),np.real(Fxkyz[1,0,0])))

In [ ]:
#---------------- Profile 
#-- Plane location
cdirection='x'; ccoords=['avg',0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='z'; ccoords=[0.02,'avg']
#----------------
plot_1D(simulation,cdirection,ccoords,
                 fieldnames=[
                     'phi',
                     ['ne','ni'],
                     'Jacobian',
                     ['upare','upari'],
                    #  'jpar',
                     ],
                 tfs=10,
                 periodicity = 0.0, grid = True
                 )

In [ ]:
#---------------- Space-time diagrams
#-- Plane location
# cdirection='x'; ccoords=[0.0,0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='ky'; ccoords=[0.01,0.0]
cdirection='z'; ccoords=[0.01,'avg']
#----------------
plot_1D_time_evolution(simulation,cdirection,ccoords,
                      #  fieldnames = ['ne','upare','Ti','phi'], # field to plot (4 moments if nothing)
                     #   fieldnames = ['Wtote','Wtoti','Wpote','Wpoti'], # field to plot (4 moments if nothing)
                       fieldnames = ['Ti','Te','ne','phi'],
                      #  twindow    = tfs[::5], space_time = True
                       twindow    = [90,95,100,105,110,120], space_time = False
                       )

In [ ]:
#---------- 2D cut plot
#-- choose min and max time frames and cut location
cplane='xy'; ccoord = 0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = tfs[-10], 
            # fieldnames = ['Wkine','Wflue','Wpote','Wtote'],
            fieldnames = ['Ti','Te','ne','phi'],
            )